## Installation

In [1]:
import nest_asyncio

nest_asyncio.apply()

## Setup API Keys

In [2]:
import os
import warnings

os.environ["LITELLM_BASE_URL"] = "https://litellm-temp-cyfjegbbfkh0buey.eastus-01.azurewebsites.net/"
os.environ["LITELLM_API_KEY"] = "sk-45309faf-a73a-4c21-a776-f10cfa92d7e1"

os.environ["ENV"] = "local"

os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

warnings.simplefilter('ignore')

## Setup Embedding model

In [3]:
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from llama_index.core import Settings

embed_model=LangchainEmbedding(
    # HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    # HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
    )
Settings.embed_model = embed_model

/tmp/ipykernel_471562/2585657273.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


## Load Data

In [4]:
from llama_index.core import SimpleDirectoryReader

#to choose specific files to load, just add the file path to the list
documents = SimpleDirectoryReader(
    input_files=["./Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx"],
    # input_files=["Datasets/medicine/Acetaminophen_Beyond_Pain_and_Fever-Relieving.txt"]

).load_data()

WHOLE_DOCUMENT = "\n\n\n<<NEW DOCUMENT>>\n\n\n".join([document.text for document in documents])

## Prompts for creating context for each chunk

We will utilize anthropic prompt caching for creating context for each chunk. If you haven’t explored our integration yet, please take a moment to review it [here](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/llm/anthropic_prompt_caching.ipynb).

In [5]:
### THIS IS WHAT ANTHROPIC ORIGINALLY WROTE. MAKE SURE WE USE CLAUDE-3.5-SONNET WITH THESE PROMPTS

# prompt_document = """<document>
# {WHOLE_DOCUMENT}
# </document>"""

# prompt_chunk = """Here is the chunk we want to situate within the whole document
# <chunk>
# {CHUNK_CONTENT}
# </chunk>
# Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else."""

In [6]:
### THIS IS WHAT I CHANGED IT TO FOR OUR TRANSCRIPTS. WE USE GEMINI-1.5-FLASH WITH THESE PROMPTS

prompt_document = """below is a transcript from which we will routinely pick excerpts to understand what they mean in the context of this document:
<document>
{WHOLE_DOCUMENT}
</document>"""

prompt_chunk = """Here is the chunk we want to situate within the whole transcript
<chunk>
{CHUNK_CONTENT}
</chunk>
Please give a short succinct context to situate this chunk within the overall transcript for the purposes of improving search retrieval of the chunk. Mention what this chunk is specifically in response to/ what this chunk talks about as a summary. Please also add a list of keywords that are relevant to searching this chunk. Answer only with the context+keywords and nothing else."""

## Utils

1. `create_contextual_nodes` - Function to create contextual nodes for a list of nodes.

2. `create_embedding_retriever` - Function to create an embedding retriever for a list of nodes.

3. `create_bm25_retriever` - Function to create a bm25 retriever for a list of nodes.

4. `create_eval_dataset` - Function to create a evaluation dataset from a list of nodes.



In [7]:
from llama_index.retrievers.bm25 import BM25Retriever

from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever
from llama_index.core.schema import NodeWithScore
from llama_index.core import VectorStoreIndex, QueryBundle

import pandas as pd
import copy
import Stemmer

from tqdm import tqdm

from typing import List

In [16]:
from utils.llm_pipeline import AsyncLLMPipelineBuilder

# llm = AsyncLLMPipelineBuilder(model="claude-3.5-sonnet", system_prompt=WHOLE_DOCUMENT)
llm = AsyncLLMPipelineBuilder(model="gemini-1.5-flash", system_prompt=WHOLE_DOCUMENT)

def create_contextual_nodes(nodes_, llm=llm, fuse_context: bool = False) -> List:
    """
    Function to create contextual nodes from a list of nodes using the LLM

    Steps:

    - take the entire document and chunk it into smaller pieces
    - send each (entire document, chunk) to the LLM to get a context for the chunk w.r.t the entire document
    - add the context to the metadata of the chunk (if fuse_context is True, add the context to the text of the chunk as well)


    Args:
        - nodes_ (List): List of nodes to create contextual nodes from
        - llm (AsyncLLMPipelineBuilder): LLM instance to use to create contextual nodes
        - fuse_context (bool): Whether to fuse the context with the text of the node. Default is False.

    Returns:
        - List: List of contextual nodes
    """

    # Create a list of messages to send to the LLM
    user_prompts = [prompt_chunk.format(CHUNK_CONTENT=node.text) for node in nodes_]

    # Send the messages to the LLM
    responses = llm.batch_predict(user_prompts)

    # Create a list of contextual nodes
    nodes_modified = []
    for node, response in zip(nodes_, responses):
        new_node = copy.deepcopy(node)
        new_node.metadata["context"] = response
        nodes_modified.append(new_node)

    # IF fuse_context is set to True, add the context to the text of the node as well as the metadata

    if fuse_context:
        for i, node in enumerate(nodes_modified):
            node.text = f"{node.metadata['context']}\n{node.text}"

    return nodes_modified

Changed model name to : gemini-1.5-flash...



In [9]:
from llama_index.core.postprocessor import SimilarityPostprocessor

def create_embedding_retriever(nodes_, similarity_top_k=5):
    """Function to create an embedding retriever for a list of nodes"""
    vector_index = VectorStoreIndex(nodes_)
    retriever = vector_index.as_retriever(similarity_top_k=similarity_top_k)
    return retriever

def create_bm25_retriever(nodes_, similarity_top_k=5):
    """Function to create a bm25 retriever for a list of nodes"""
    bm25_retriever = BM25Retriever.from_defaults(
        nodes=nodes_,
        similarity_top_k=similarity_top_k,
        stemmer=Stemmer.Stemmer("english"),
        language="english",
    )
    return bm25_retriever

def set_node_ids(nodes_):
    """Function to set node ids for a list of nodes"""

    # by default, the node ids are set to random uuids. To ensure same id's per run, we manually set them.
    for index, node in enumerate(nodes_):
        node.id_ = f"node_{index}"

    return nodes_

In [10]:
class EmbeddingRetriever(BaseRetriever):
    """Custom retriever that uses embedding retriever"""

    def __init__(self,
                 vector_retriever: VectorIndexRetriever,
                 similarity_cutoff:float=0.25,
                 debug:bool=False) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self.node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)]

        self.debug = debug

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query, postprocesses them to keep only those that satisfy minimum similarity."""

        all_retrieved_nodes = self._vector_retriever.retrieve(query_bundle)

        # postprocess nodes
        relevant_nodes = self.node_postprocessors[0]._postprocess_nodes(nodes=all_retrieved_nodes,
                                                                     query_bundle=query_bundle)
        
        if self.debug:
            print(f"Number of nodes retrieved : {len(all_retrieved_nodes)} | Number of relevant nodes: {len(relevant_nodes)}")
            print(f"sample retrieved node : {all_retrieved_nodes[-1]}\nsample relevant node : {relevant_nodes[-1]}")
        
        return relevant_nodes
    
    def retrieve_results_for_query(self, query:str):
        """
        Retrieve results for a query - used for testing purposes
        """

        query_bundle = QueryBundle(query)
        return self._retrieve(query_bundle)
    
class EmbeddingBM25Retriever(BaseRetriever):
    """Custom retriever that uses both embedding and bm25 retrievers"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        bm25_retriever: BM25Retriever,
        similarity_cutoff:float=0.25,
        debug:bool=False
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        self.node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)]

        self.debug = debug

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        bm25_nodes = self.bm25_retriever.retrieve(query_bundle)

        vector_nodes.extend(bm25_nodes)

        relevant_nodes = self.node_postprocessors[0]._postprocess_nodes(
            vector_nodes, query_bundle
        )

        if self.debug:
            print(f"Number of nodes retrieved : {len(vector_nodes)} | Number of relevant nodes: {len(relevant_nodes)}")
            print(f"sample vector node : {vector_nodes[-1]}\nsample reranked node : {relevant_nodes[-1]}")

        return relevant_nodes
    
    def retrieve_results_for_query(self, query:str):
        """
        Retrieve results for a query - used for testing purposes
        """

        query_bundle = QueryBundle(query)
        return self._retrieve(query_bundle)
    
class EmbeddingBM25RerankerRetriever(BaseRetriever):
    """Custom retriever that uses both embedding and bm25 retrievers, and reranks the results"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        bm25_retriever: BM25Retriever,
        reranker=None,
        debug:bool=False,
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        self.reranker = reranker

        self.debug = debug

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        bm25_nodes = self.bm25_retriever.retrieve(query_bundle)

        vector_nodes.extend(bm25_nodes)

        retrieved_nodes = self.reranker.postprocess_nodes(
            vector_nodes, query_bundle
        )

        return retrieved_nodes

        # return vector_nodes
    
    def retrieve_results_for_query(self, query:str):
        """
        Retrieve results for a query - used for testing purposes
        """

        query_bundle = QueryBundle(query)
        return self._retrieve(query_bundle)

## Create Nodes

In [11]:
from llama_index.core.node_parser import SentenceSplitter

#keep playing with different embedding models and chunk sizes to see what works best

node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
# node_parser = SentenceSplitter(chunk_size=400, chunk_overlap=80)

nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

## Set node ids

Useful to have consistent result comparison for nodes with and without contextual text.

In [12]:
# set node ids

nodes = set_node_ids(nodes)

In [13]:
nodes[0].metadata

{'file_path': 'Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx',
 'file_name': 'v2t_SpeakerResponse_cleaned.xlsx',
 'file_type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'file_size': 37698,
 'creation_date': '2024-10-27',
 'last_modified_date': '2024-10-27'}

In [14]:
print(f"split data into {len(nodes)} nodes")

split data into 23 nodes


## Create contextual nodes -- anthropic's "contextual retrieval" approach

In [15]:
# nodes_contextual = create_contextual_nodes(nodes)
nodes_contextual = create_contextual_nodes(nodes, fuse_context=True)

  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 23 requests either timed out or returned non-parseable outputs ...


100%|██████████| 1/1 [00:06<00:00,  6.80s/it]


In [17]:
nodes[0].metadata, nodes_contextual[0].metadata

({'file_path': 'Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx',
  'file_name': 'v2t_SpeakerResponse_cleaned.xlsx',
  'file_type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
  'file_size': 37698,
  'creation_date': '2024-10-27',
  'last_modified_date': '2024-10-27'},
 {'file_path': 'Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx',
  'file_name': 'v2t_SpeakerResponse_cleaned.xlsx',
  'file_type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
  'file_size': 37698,
  'creation_date': '2024-10-27',
  'last_modified_date': '2024-10-27',
  'context': 'Context: This chunk is the beginning of the interview, where the interviewers introduce themselves and their company, Clear Strategy, and explain the purpose of the interview, which is to gather customer feedback on fast food. They also assure the interviewee that their information will be kept confidential and anonymous. \n\nKeywords: marketing research, Clear Strategy, customer 

<!-- ## Set `CohereReranker` -->
## Set `ColbertReranker`

- set limits for number of documents to retrieve/similarity of documents etc.

In [18]:
similarity_top_k = 4 #number of matching nodes. PROTIP : increase this IF the chunk size is small. Decrease if the chunk size is large
similarity_top_k_after_reranking = 4 #number of matching nodes after reranking. PROTIP : increase this IF the chunk size is small. Decrease if the chunk size is large
similarity_cutoff = 0.4 #minimum similarity score for a node to be considered relevant

In [19]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=similarity_top_k_after_reranking,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

## Create retrievers.

1. Embedding based retriever.
2. BM25 based retriever.
3. Embedding + BM25 + Colbert reranker retriever.

## Create retrievers using contextual nodes.

In [20]:
contextual_embedding_retriever = create_embedding_retriever(
    nodes_contextual, similarity_top_k=similarity_top_k
)
contextual_bm25_retriever = create_bm25_retriever(
    nodes_contextual, similarity_top_k=similarity_top_k
)

#make them all class objects for retrieval testing

contextual_embedding_bm25_retriever_rerank = EmbeddingBM25RerankerRetriever(
    contextual_embedding_retriever,
    contextual_bm25_retriever,
    reranker=colbert_reranker,
)
contextual_embedding_retriever = EmbeddingRetriever(
    contextual_embedding_retriever, similarity_cutoff=similarity_cutoff
)
contextual_embedding_bm25_retriever = EmbeddingBM25Retriever(
    contextual_embedding_retriever, contextual_bm25_retriever, similarity_cutoff=similarity_cutoff
)

## Define simple functions to retrieve relevant results from queries

In [21]:
def get_relevant_chunks(
                        query:str,
                        retriever=contextual_embedding_bm25_retriever):
    """
    Retrieve relevant chunks for a query

    Args:
        - query : str : query string
        - retriever : BaseRetriever : retriever object to use for retrieval. Defaults to contextual_embedding_bm25_retriever, which uses both embedding and bm25 retrievers

    Returns:
        - List[NodeWithScore] : list of relevant nodes with scores
    """
    
    return retriever.retrieve_results_for_query(query)

In [22]:
def answer_query(relevant_chunks, query, llm, use_context:bool=False):
    """Function to answer a query given relevant chunks
    
    Args:
        - relevant_chunks : List[NodeWithScore] : list of relevant nodes with scores
        - query : str : query string
        - llm : AsyncLLMPipelineBuilder : llm object to use for answering the query
        - use_context : bool : whether to use context for the text excerpts. Defaults to False

    Returns:
        - str : formatted chunks to be sent to the LLM. This is a string with all the relevant chunks formatted as per the LLM prompt
        - str : response from the LLM
    """

    context = None
    if use_context:
        try:
            context = relevant_chunks[-1].metadata['context']

            #TODO : add a ReAct prompt or a chain-of-thought prompt to basically make it [look at context/keywords of a chunk -> decide if it wants to use this to answer the query -> answer the query]
            
        except Exception as e:
            print(e)
            pass

    # Create a list of messages to send to the LLM
    answering_prompt = """
                        Answer the query : `{query}` using any of the relevant pieces of texts below:
                        
                        {CHUNKS}

                        Your answer should be precise and relevant to the query. Note that the text excerpts are provided to help you answer the question, but may contain a lot of irrelevant information. Use only the necessary information to answer the query appropriately.
                        """
    
    #send all chunks as a single string to be answered
    CHUNKS = [chunk.text for chunk in relevant_chunks]
    CHUNKS = list(set(CHUNKS))
    CHUNKS = "\n".join(CHUNKS)
    
    if context is not None:
        CHUNKS = [f"use this context for the text excerpt : {chunk.metadata['context']}\ntext excerpt : {chunk.text}\n"
                  for chunk in relevant_chunks]
        CHUNKS = list(set(CHUNKS))
        CHUNKS = "\n".join(CHUNKS)
    
    user_prompts = [answering_prompt.format(query=query, CHUNKS=CHUNKS)]

    # Send the messages to the LLM
    response = llm.batch_predict(user_prompts)[-1]

    return CHUNKS, response

def retrieve_relevant_chunks_and_answer_query(query:str,
                                              llm,
                                              retriever=contextual_embedding_bm25_retriever,
                                              answer_with_context:bool=False):
    """
    Retrieve relevant chunks and answer a query

    Args:
        - query : str : query string
        - llm : AsyncLLMPipelineBuilder : llm object to use for answering the query
        - retriever : BaseRetriever : retriever object to use for retrieval. Defaults to contextual_embedding_bm25_retriever, which uses both embedding and bm25 retrievers
        - answer_with_context : bool : whether to use context for the text excerpts. Defaults to False

    Returns:
        - dict : dictionary with the query, documents retrieved, chunks formatted for answering, and llm response
    """

    relevant_chunks = get_relevant_chunks(query, retriever)
    CHUNKS_USED_FOR_LLM_QUERY, response = answer_query(relevant_chunks, query, llm, use_context=answer_with_context)

    return {"query" : query,
            "documents retrieved" : relevant_chunks,
            "chunks formatted for answering" : CHUNKS_USED_FOR_LLM_QUERY,
            "llm_response" : response}

In [24]:
query = "what is the most preferred side dish? be descriptive"
# query = "what is APAP? Explain in detail"
relevant_chunks = get_relevant_chunks(query=query)
relevant_chunks

[NodeWithScore(node=TextNode(id_='node_4', embedding=None, metadata={'file_path': 'Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx', 'file_name': 'v2t_SpeakerResponse_cleaned.xlsx', 'file_type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'file_size': 37698, 'creation_date': '2024-10-27', 'last_modified_date': '2024-10-27', 'context': "Context: This chunk is a continuation of the discussion about the participant's preference for KFC chicken over McDonald's chicken. The participant explains that KFC chicken is more of a main dish for them, while McDonald's chicken is more of a side dish. The conversation then transitions to the participant's experience with the McCrispy chicken, which they have tried recently. \n\nKeywords: KFC, McDonald's, chicken, McCrispy, preference, main dish, side dish, taste, fragrance, spicy, marination, batter \n"}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_a

## Define an LLM to see what kind of `RAG` can be built with the retrieved documents

#### @TODO : Answering LLM for this RAG should make full use of the context/keywords provided. Define some sort of `chain-of-thought` reasoning or `ReAct framework` reasoning for it

In [25]:
rag_llm = AsyncLLMPipelineBuilder(""""You are an expert analyst chatbot named EzyChat. you will be given excepts from a transcript
                                    and a question, you will have to answer the question solely based on the
                                    information provided in the excepts.
                                    Do not make generalisations or assumptions.
                                    Be confident in your answers. but also helpful
                                    Do not answer any questions that are not in scope of the given excerpts.
                                    Do not mention the excerpt from which your answer is from.""",
                                    # model="gemini-1.5-flash"
                                    # model="gpt-4o"
                                    # model="gemini-1.5-pro"
                                    model="claude-3.5-sonnet"
                                )

Changed model name to : claude-3.5-sonnet...



In [27]:
answer_query(relevant_chunks,
             query=query,
             llm=rag_llm, use_context=True)

  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


100%|██████████| 1/1 [00:06<00:00,  6.72s/it]


("use this context for the text excerpt : Context: This chunk is a continuation of the discussion about the participant's preference for KFC chicken over McDonald's chicken. The participant explains that KFC chicken is more of a main dish for them, while McDonald's chicken is more of a side dish. The conversation then transitions to the participant's experience with the McCrispy chicken, which they have tried recently. \n\nKeywords: KFC, McDonald's, chicken, McCrispy, preference, main dish, side dish, taste, fragrance, spicy, marination, batter \n\ntext excerpt : Context: This chunk is a continuation of the discussion about the participant's preference for KFC chicken over McDonald's chicken. The participant explains that KFC chicken is more of a main dish for them, while McDonald's chicken is more of a side dish. The conversation then transitions to the participant's experience with the McCrispy chicken, which they have tried recently. \n\nKeywords: KFC, McDonald's, chicken, McCrispy,

In [29]:
retrieve_relevant_chunks_and_answer_query(
                                            query="Compare the opinions regarding liking Bone-in meat versus thigh/thai meat",
                                            # query = "What is the most uncommon use of acetoaminophen?",
                                            llm=rag_llm,
                                            # retriever=contextual_embedding_bm25_retriever,
                                            # retriever=contextual_embedding_retriever,
                                            retriever=contextual_embedding_bm25_retriever_rerank,
                                            answer_with_context=True)

  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


100%|██████████| 1/1 [00:05<00:00,  5.87s/it]


{'query': 'Compare the opinions regarding liking Bone-in meat versus thigh/thai meat',
 'documents retrieved': [NodeWithScore(node=TextNode(id_='node_8', embedding=None, metadata={'file_path': 'Datasets/chicken_data/v2t_SpeakerResponse_cleaned.xlsx', 'file_name': 'v2t_SpeakerResponse_cleaned.xlsx', 'file_type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'file_size': 37698, 'creation_date': '2024-10-27', 'last_modified_date': '2024-10-27', 'context': 'This chunk is a discussion between the interviewer and interviewee about what could make the McCrispy chicken more appealing to the interviewee. The interviewee suggests a value box with McCrispy chicken as a patty, tenders, and nuggets. They also discuss the importance of whole meat versus minced meat in chicken products. \n\nKeywords: McCrispy, chicken, value box, patty, tenders, nuggets, whole meat, minced meat \n', 'retrieval_score': 5.351810932159424}, excluded_embed_metadata_keys=['file_name', 'file_type', '

In [31]:
questions="""What Reasons to try chicken are mentioned?
what do the participants mention as Expectations from chicken?
Compare the opinions regarding liking Bone-in meat versus thigh/thai meat
What are some new products and innovations desired in chicken
What is the average Experience at McDonald?
What is the average Experience at KFC?
Give me a Persona mapping of Brands. answer as bullet points
What items/products of the menu are Easy to consume?
Which items are associated with great fragrance/great smell/great aroma?
What are some good sources of protein mentioned in the data?
What are the main themes discussed in this conversation? List in bullet points
Compare McDonalds with KFC in terms of their taste and range of products""".split("\n")

# questions = """What is the major concern with acetoaminophen?
# What to avoid when taking acetoaminophen?
# What do you assess before taking acetoaminophen?
# What is the most common use of acetoaminophen?
# What is the most uncommon use of acetoaminophen?
# Why is acetaminophen not recommended for children under 6 months?
# What organs are affected by acetaminophen?""".split("\n")

questions

['What Reasons to try chicken are mentioned?',
 'what do the participants mention as Expectations from chicken?',
 'Compare the opinions regarding liking Bone-in meat versus thigh/thai meat',
 'What are some new products and innovations desired in chicken',
 'What is the average Experience at McDonald?',
 'What is the average Experience at KFC?',
 'Give me a Persona mapping of Brands. answer as bullet points',
 'What items/products of the menu are Easy to consume?',
 'Which items are associated with great fragrance/great smell/great aroma?',
 'What are some good sources of protein mentioned in the data?',
 'What are the main themes discussed in this conversation? List in bullet points',
 'Compare McDonalds with KFC in terms of their taste and range of products']

In [32]:
responses = []

for question in questions:
    try:
        responses.append(retrieve_relevant_chunks_and_answer_query(query=question,
                                                               llm=rag_llm,
                                                            #    retriever=contextual_embedding_bm25_retriever,
                                                            #    retriever=contextual_embedding_retriever,
                                                               retriever=contextual_embedding_bm25_retriever_rerank,
                                                               answer_with_context=False)
                        )
        
    except Exception as e:
        print(e)
        responses.append({"query" : question,
                          "documents retrieved" : None,
                          "chunks formatted for answering" : None,
                          "llm_response" : str(e)})
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


  0%|          | 0/1 [00:00<?, ?it/s]

0 requests out of 1 requests either timed out or returned non-parseable outputs ...


100%|██████████| 1/1 [00:06<00:00,  6.66s/it]


In [33]:
pd.DataFrame(responses)
# .to_excel("./Outputs/chicken_data_llamaindex_contextual_bm25_retrieval_rerank.xlsx", index=False)

,query,documents retrieved,chunks formatted for answering,llm_response
0,What Reasons to try chicken are mentioned?,[Node ID: node_11\nText: This chunk is a discu...,This chunk is a discussion about how to make c...,"Based on the provided excerpts, some reasons t..."
1,what do the participants mention as Expectatio...,[Node ID: node_16\nText: Context: This chunk i...,Context: This chunk is a continuation of the d...,"Based on the provided excerpts, the participan..."
2,Compare the opinions regarding liking Bone-in ...,[Node ID: node_8\nText: This chunk is a discus...,This chunk is a discussion about the consumer'...,"Based on the provided excerpts, there is no di..."
3,What are some new products and innovations des...,[Node ID: node_16\nText: Context: This chunk i...,This chunk is a discussion about the responden...,"Based on the provided excerpts, some new produ..."
4,What is the average Experience at McDonald?,[Node ID: node_16\nText: Context: This chunk i...,Context: This chunk is a continuation of the d...,"I apologize, but I cannot provide an average e..."
5,What is the average Experience at KFC?,[Node ID: node_16\nText: Context: This chunk i...,Context: This chunk is a continuation of the d...,"I apologize, but I cannot provide an answer to..."
6,Give me a Persona mapping of Brands. answer as...,[Node ID: node_0\nText: Context: This chunk is...,Context: This chunk is a discussion about the ...,Here's a persona mapping of fast food chicken ...
7,What items/products of the menu are Easy to co...,[Node ID: node_20\nText: This chunk is a discu...,This chunk is a discussion about the consumer'...,Based on the information provided in the excer...
8,Which items are associated with great fragranc...,[Node ID: node_5\nText: Context: This chunk is...,Context: This chunk is a continuation of the d...,"Based on the provided excerpts, there is no sp..."
9,What are some good sources of protein mentione...,[Node ID: node_11\nText: This chunk is a discu...,Context: This chunk is a continuation of the d...,"Based on the provided excerpts, some good sour..."
